# Import Libraries

In [1]:
import requests # to handle requests
import pandas as pd # for data analsysis
import numpy as np # to handle data in a vectorized manner

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
#tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



# Foursuare Credentials

In [2]:

CLIENT_ID = 'GEOB3Q4RTF3RYS1WW0YUO1K1TH2S3JACF40KFZGGILESXARV'
CLIENT_SECRET = 'DZT1HG4VZQFFQVQRZLIVQ25HFWJAEAWIEKKO2AYGE1HHOB3J'
VERSION = '20200212' 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GEOB3Q4RTF3RYS1WW0YUO1K1TH2S3JACF40KFZGGILESXARV
CLIENT_SECRET:DZT1HG4VZQFFQVQRZLIVQ25HFWJAEAWIEKKO2AYGE1HHOB3J


# Define city and its latitude & longitude

In [3]:
city = 'Philadelphia'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(city)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

39.9527237 -75.1635262


# Search for Hotels

In [4]:
# search for hotels
search_query = 'Hotel'
radius = 500

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}'\
.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius)
url

'https://api.foursquare.com/v2/venues/search?client_id=GEOB3Q4RTF3RYS1WW0YUO1K1TH2S3JACF40KFZGGILESXARV&client_secret=DZT1HG4VZQFFQVQRZLIVQ25HFWJAEAWIEKKO2AYGE1HHOB3J&ll=39.9527237,-75.1635262&v=20200212&query=Hotel&radius=500'

In [5]:

# Send the GET Request and examine the results
results = requests.get(url).json()
#results

In [6]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,...,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",NaN,NaN,NaN,NaN,NaN,NaN,False,5d30fd7d844dd800083fe25c,21 N Juniper St,...,"[21 N Juniper St, Philadelphia, PA 19107, Unit...","[{'label': 'display', 'lat': 39.953016, 'lng':...",39.953016,-75.161805,NaN,19107,PA,The Notary Hotel,v-1581876981,NaN
1,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",NaN,NaN,NaN,NaN,NaN,NaN,False,4a0bf2c3f964a5200e751fe3,1200 Market St,...,"[1200 Market St, Philadelphia, PA 19107, Unite...","[{'label': 'display', 'lat': 39.95141891142559...",39.951419,-75.160303,NaN,19107,PA,Loews Philadelphia Hotel,v-1581876981,34285545
2,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",NaN,NaN,NaN,NaN,NaN,NaN,False,4afb8f59f964a520a11e22e3,117 S 17th St,...,"[117 S 17th St (at Sansom St), Philadelphia, P...","[{'label': 'display', 'lat': 39.950809, 'lng':...",39.950809,-75.168642,NaN,19103,PA,Kimpton Hotel Palomar Philadelphia,v-1581876981,44568656
3,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",NaN,NaN,NaN,NaN,NaN,NaN,False,4aba8dbff964a520d78120e3,1628 Chestnut St,...,"[1628 Chestnut St (at 17th St.), Philadelphia,...","[{'label': 'display', 'lat': 39.95130770193616...",39.951308,-75.168396,NaN,19103,PA,Club Quarters Hotel in Philadelphia,v-1581876981,NaN
4,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",1523643,/delivery_provider_grubhub_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",grubhub,https://www.grubhub.com/restaurant/basement-at...,False,49d57ecef964a520985c1fe3,200 S Broad St,...,"[200 S Broad St, Philadelphia, PA 19102, Unite...","[{'label': 'display', 'lat': 39.94905, 'lng': ...",39.949050,-75.164850,NaN,19102,PA,The Bellevue Hotel,v-1581876981,NaN


# Clean Hotel Dataframe

In [7]:
# keep only columns that include venue name, and anything that is associated with location
clean_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')]+ ['id']
clean_dataframe = dataframe.loc[:,clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
clean_dataframe['categories'] = clean_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_dataframe.columns = [column.split('.')[-1] for column in clean_dataframe.columns]

clean_dataframe.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,The Notary Hotel,Hotel,21 N Juniper St,US,Philadelphia,United States,NaN,150,"[21 N Juniper St, Philadelphia, PA 19107, Unit...","[{'label': 'display', 'lat': 39.953016, 'lng':...",39.953016,-75.161805,NaN,19107,PA,5d30fd7d844dd800083fe25c
1,Loews Philadelphia Hotel,Hotel,1200 Market St,US,Philadelphia,United States,NaN,311,"[1200 Market St, Philadelphia, PA 19107, Unite...","[{'label': 'display', 'lat': 39.95141891142559...",39.951419,-75.160303,NaN,19107,PA,4a0bf2c3f964a5200e751fe3
2,Kimpton Hotel Palomar Philadelphia,Hotel,117 S 17th St,US,Philadelphia,United States,at Sansom St,485,"[117 S 17th St (at Sansom St), Philadelphia, P...","[{'label': 'display', 'lat': 39.950809, 'lng':...",39.950809,-75.168642,NaN,19103,PA,4afb8f59f964a520a11e22e3
3,Club Quarters Hotel in Philadelphia,Hotel,1628 Chestnut St,US,Philadelphia,United States,at 17th St.,444,"[1628 Chestnut St (at 17th St.), Philadelphia,...","[{'label': 'display', 'lat': 39.95130770193616...",39.951308,-75.168396,NaN,19103,PA,4aba8dbff964a520d78120e3
4,The Bellevue Hotel,Hotel,200 S Broad St,US,Philadelphia,United States,NaN,424,"[200 S Broad St, Philadelphia, PA 19102, Unite...","[{'label': 'display', 'lat': 39.94905, 'lng': ...",39.949050,-75.164850,NaN,19102,PA,49d57ecef964a520985c1fe3


In [8]:
# delete unnecessary columns
clean_dataframe2= clean_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs','neighborhood', 'id'], axis=1)
clean_dataframe2

,name,categories,address,lat,lng,postalCode,state
0,The Notary Hotel,Hotel,21 N Juniper St,39.953016,-75.161805,19107,PA
1,Loews Philadelphia Hotel,Hotel,1200 Market St,39.951419,-75.160303,19107,PA
2,Kimpton Hotel Palomar Philadelphia,Hotel,117 S 17th St,39.950809,-75.168642,19103,PA
3,Club Quarters Hotel in Philadelphia,Hotel,1628 Chestnut St,39.951308,-75.168396,19103,PA
4,The Bellevue Hotel,Hotel,200 S Broad St,39.949050,-75.164850,19102,PA
5,Cambria Hotel Philadelphia Downtown Center City,Hotel,219 S Broad St,39.948627,-75.164198,19107,PA
6,The Independent Hotel,Hotel,1234 Locust St,39.947781,-75.162146,19107,PA
7,Harleysville hotel,American Restaurant,Main street,39.954462,-75.162617,NaN,PA
8,Philadelphia 201 Hotel,Hotel,NaN,39.956928,-75.167478,19103,PA
9,Hotel California,None,NaN,39.951400,-75.165598,NaN,Pennsylvania


In [9]:

# delete rows with none values
clean_dataframe3 = clean_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
clean_dataframe3

,name,categories,address,lat,lng,postalCode,state
0,The Notary Hotel,Hotel,21 N Juniper St,39.953016,-75.161805,19107,PA
1,Loews Philadelphia Hotel,Hotel,1200 Market St,39.951419,-75.160303,19107,PA
2,Kimpton Hotel Palomar Philadelphia,Hotel,117 S 17th St,39.950809,-75.168642,19103,PA
3,Club Quarters Hotel in Philadelphia,Hotel,1628 Chestnut St,39.951308,-75.168396,19103,PA
4,The Bellevue Hotel,Hotel,200 S Broad St,39.949050,-75.164850,19102,PA
5,Cambria Hotel Philadelphia Downtown Center City,Hotel,219 S Broad St,39.948627,-75.164198,19107,PA
6,The Independent Hotel,Hotel,1234 Locust St,39.947781,-75.162146,19107,PA
11,The Solstice Room at Loews Philadelphia Hotel,Hotel,1200 Market St,39.951797,-75.159876,19107,PA
18,Philadelphia Marriott Downtown,Hotel,"1201 Market Street, (Guest Entrance At 1200 Fi...",39.952604,-75.160625,19107,PA
19,Franklin Room Westin Hotel,Meeting Room,99 S 17th St,39.951601,-75.168521,19103,PA


In [10]:
# delete rows which its category is not Hotel or Event Space
array= ['Hotel', 'Event Space']
hotel_dataframe= clean_dataframe3.loc[clean_dataframe3['categories'].isin(array)]
hotel_dataframe

,name,categories,address,lat,lng,postalCode,state
0,The Notary Hotel,Hotel,21 N Juniper St,39.953016,-75.161805,19107,PA
1,Loews Philadelphia Hotel,Hotel,1200 Market St,39.951419,-75.160303,19107,PA
2,Kimpton Hotel Palomar Philadelphia,Hotel,117 S 17th St,39.950809,-75.168642,19103,PA
3,Club Quarters Hotel in Philadelphia,Hotel,1628 Chestnut St,39.951308,-75.168396,19103,PA
4,The Bellevue Hotel,Hotel,200 S Broad St,39.949050,-75.164850,19102,PA
5,Cambria Hotel Philadelphia Downtown Center City,Hotel,219 S Broad St,39.948627,-75.164198,19107,PA
6,The Independent Hotel,Hotel,1234 Locust St,39.947781,-75.162146,19107,PA
11,The Solstice Room at Loews Philadelphia Hotel,Hotel,1200 Market St,39.951797,-75.159876,19107,PA
18,Philadelphia Marriott Downtown,Hotel,"1201 Market Street, (Guest Entrance At 1200 Fi...",39.952604,-75.160625,19107,PA
20,Bingham House Hotel,Hotel,11th and Market Street,39.951713,-75.158306,19107,PA


In [11]:

# delete rows which has duplicate hotel's name
df_hotels = hotel_dataframe.drop_duplicates(subset='name', keep="first")
df_hotels

,name,categories,address,lat,lng,postalCode,state
0,The Notary Hotel,Hotel,21 N Juniper St,39.953016,-75.161805,19107,PA
1,Loews Philadelphia Hotel,Hotel,1200 Market St,39.951419,-75.160303,19107,PA
2,Kimpton Hotel Palomar Philadelphia,Hotel,117 S 17th St,39.950809,-75.168642,19103,PA
3,Club Quarters Hotel in Philadelphia,Hotel,1628 Chestnut St,39.951308,-75.168396,19103,PA
4,The Bellevue Hotel,Hotel,200 S Broad St,39.949050,-75.164850,19102,PA
5,Cambria Hotel Philadelphia Downtown Center City,Hotel,219 S Broad St,39.948627,-75.164198,19107,PA
6,The Independent Hotel,Hotel,1234 Locust St,39.947781,-75.162146,19107,PA
11,The Solstice Room at Loews Philadelphia Hotel,Hotel,1200 Market St,39.951797,-75.159876,19107,PA
18,Philadelphia Marriott Downtown,Hotel,"1201 Market Street, (Guest Entrance At 1200 Fi...",39.952604,-75.160625,19107,PA
20,Bingham House Hotel,Hotel,11th and Market Street,39.951713,-75.158306,19107,PA


# Search for Grocery Store

In [12]:
search_query = 'grocery store'
radius = 10000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}'\
.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius,)
url

'https://api.foursquare.com/v2/venues/search?client_id=GEOB3Q4RTF3RYS1WW0YUO1K1TH2S3JACF40KFZGGILESXARV&client_secret=DZT1HG4VZQFFQVQRZLIVQ25HFWJAEAWIEKKO2AYGE1HHOB3J&ll=39.9527237,-75.1635262&v=20200212&query=grocery store&radius=10000'

In [13]:
# Send the GET Request and examine the results
presults = requests.get(url).json()
#presults

In [14]:
# assign relevant part of JSON to venues
venues = presults['response']['venues']

# tranform venues into a dataframe
grocerystore_dataframe = json_normalize(venues)
grocerystore_dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",False,4b61db1ef964a52057272ae3,1227 S 18th St,US,Philadelphia,United States,btw 18th and Latona,2071,"[1227 S 18th St (btw 18th and Latona), Philade...","[{'label': 'display', 'lat': 39.93584892968741...",39.935849,-75.173754,19146,PA,Fernandez Grocery Store,v-1581876981
1,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",False,4da8cef80cb6a89c624731bd,2200 Benjamin Franklin Pkwy,US,Philadelphia,United States,NaN,1217,"[2200 Benjamin Franklin Pkwy, Philadelphia, PA...","[{'label': 'display', 'lat': 39.958787, 'lng':...",39.958787,-75.175407,19130,PA,Grocery Store,v-1581876981
2,"[{'id': '4d954b0ea243a5684a65b473', 'name': 'C...",False,55cb5f4f498ea925f1c61718,NaN,US,Philadelphia,United States,NaN,1699,"[Philadelphia, PA, United States]","[{'label': 'display', 'lat': 39.93849786664969...",39.938498,-75.156285,NaN,PA,Lucky 7 Grocery Store,v-1581876981
3,"[{'id': '4bf58dd8d48988d1f9941735', 'name': 'F...",False,4b5a21d7f964a5203daf28e3,1500 S Broad St,US,Philadelphia,United States,15th and Dickinson,2381,"[1500 S Broad St (15th and Dickinson), Philade...","[{'label': 'display', 'lat': 39.93167299999999...",39.931673,-75.168482,19146,PA,88 Grocery Store,v-1581876981
4,"[{'id': '4bf58dd8d48988d146941735', 'name': 'D...",False,572f6a31498e66d843d0c2d6,NaN,US,Philadelphia,United States,NaN,1981,"[Philadelphia, PA, United States]","[{'label': 'display', 'lat': 39.934977, 'lng':...",39.934977,-75.161750,NaN,PA,Edith Grocery Store,v-1581876981


# Clean Park Dataframe

In [15]:
# keep only columns that include venue name, and anything that is associated with location
grocerystore_clean_columns = ['name', 'categories'] + [col for col in grocerystore_dataframe.columns if col.startswith('location.')]+ ['id']
clean_grocerystore_dataframe = grocerystore_dataframe.loc[:,grocerystore_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list1 = row['categories']
    except:
        categories_list1 = row['venue.categories']
        
    if len(categories_list1) == 0:
        return None
    else:
        return categories_list1[0]['name']

# filter the category for each row
clean_grocerystore_dataframe['categories'] = clean_grocerystore_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_grocerystore_dataframe.columns = [column.split('.')[-1] for column in clean_grocerystore_dataframe.columns]

clean_grocerystore_dataframe.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Fernandez Grocery Store,Grocery Store,1227 S 18th St,US,Philadelphia,United States,btw 18th and Latona,2071,"[1227 S 18th St (btw 18th and Latona), Philade...","[{'label': 'display', 'lat': 39.93584892968741...",39.935849,-75.173754,19146,PA,4b61db1ef964a52057272ae3
1,Grocery Store,Grocery Store,2200 Benjamin Franklin Pkwy,US,Philadelphia,United States,NaN,1217,"[2200 Benjamin Franklin Pkwy, Philadelphia, PA...","[{'label': 'display', 'lat': 39.958787, 'lng':...",39.958787,-75.175407,19130,PA,4da8cef80cb6a89c624731bd
2,Lucky 7 Grocery Store,Convenience Store,NaN,US,Philadelphia,United States,NaN,1699,"[Philadelphia, PA, United States]","[{'label': 'display', 'lat': 39.93849786664969...",39.938498,-75.156285,NaN,PA,55cb5f4f498ea925f1c61718
3,88 Grocery Store,Food & Drink Shop,1500 S Broad St,US,Philadelphia,United States,15th and Dickinson,2381,"[1500 S Broad St (15th and Dickinson), Philade...","[{'label': 'display', 'lat': 39.93167299999999...",39.931673,-75.168482,19146,PA,4b5a21d7f964a5203daf28e3
4,Edith Grocery Store,Deli / Bodega,NaN,US,Philadelphia,United States,NaN,1981,"[Philadelphia, PA, United States]","[{'label': 'display', 'lat': 39.934977, 'lng':...",39.934977,-75.161750,NaN,PA,572f6a31498e66d843d0c2d6


In [16]:
# delete unnecessary columns
clean_grocerystore_dataframe2= clean_grocerystore_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'id'], axis=1)
clean_grocerystore_dataframe2

,name,categories,address,lat,lng,postalCode,state
0,Fernandez Grocery Store,Grocery Store,1227 S 18th St,39.935849,-75.173754,19146,PA
1,Grocery Store,Grocery Store,2200 Benjamin Franklin Pkwy,39.958787,-75.175407,19130,PA
2,Lucky 7 Grocery Store,Convenience Store,NaN,39.938498,-75.156285,NaN,PA
3,88 Grocery Store,Food & Drink Shop,1500 S Broad St,39.931673,-75.168482,19146,PA
4,Edith Grocery Store,Deli / Bodega,NaN,39.934977,-75.161750,NaN,PA
5,Luna Grocery Store,Convenience Store,NaN,39.934820,-75.161963,NaN,Pennsylvania
6,L. M. Grocery Store,Convenience Store,NaN,39.933393,-75.162310,NaN,Pennsylvania
7,Eak Meng: Cambodian Grocery Store,Grocery Store,7th Street,39.922412,-75.158218,19148,PA
8,Sun Grocery Store,Convenience Store,NaN,39.924047,-75.175087,NaN,PA
9,Lin's Grocery Store,Grocery Store,NaN,39.922664,-75.152295,NaN,Pennsylvania


In [17]:
# delete rows with none values
clean_grocerystore_dataframe3 = clean_grocerystore_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
clean_grocerystore_dataframe3

,name,categories,address,lat,lng,postalCode,state
0,Fernandez Grocery Store,Grocery Store,1227 S 18th St,39.935849,-75.173754,19146,PA
1,Grocery Store,Grocery Store,2200 Benjamin Franklin Pkwy,39.958787,-75.175407,19130,PA
3,88 Grocery Store,Food & Drink Shop,1500 S Broad St,39.931673,-75.168482,19146,PA
7,Eak Meng: Cambodian Grocery Store,Grocery Store,7th Street,39.922412,-75.158218,19148,PA
10,Jerusalem Grocery Store,Middle Eastern Restaurant,115 W Girard Ave,39.969112,-75.136764,19123,PA
11,Venuti's Grocery Store,Grocery Store,South Camac St.,39.916994,-75.168181,19148,PA
13,10th & Snyder Grocery Store,Grocery Store,1000 Snyder Ave,39.923217,-75.162336,19148,PA
14,Grocery Stores,Mexican Restaurant,17th Street,39.964811,-75.165907,19130,PA
15,Anna Grocery Store,Deli / Bodega,2200 S 20th St,39.924285,-75.179552,19145,PA
16,Andrew Grocery Store,Deli / Bodega,400 McKean Street,39.923715,-75.153132,19146,PA


# Search for Restaurants

In [18]:
# search for Restaurants
search_query = 'Restaurant'
radius = 10000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius)
url

'https://api.foursquare.com/v2/venues/search?client_id=GEOB3Q4RTF3RYS1WW0YUO1K1TH2S3JACF40KFZGGILESXARV&client_secret=DZT1HG4VZQFFQVQRZLIVQ25HFWJAEAWIEKKO2AYGE1HHOB3J&ll=39.9527237,-75.1635262&v=20200212&query=Restaurant&radius=10000'

In [19]:

# Send the GET Request and examine the results
Rresults = requests.get(url).json()
#Rresults

In [20]:

# assign relevant part of JSON to venues
venues = Rresults['response']['venues']

# tranform venues into a dataframe
Restaurant_dataframe = json_normalize(venues)
Restaurant_dataframe.head()

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,...,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d11b941735', 'name': 'P...",NaN,NaN,NaN,NaN,NaN,NaN,False,40b28c80f964a52033f91ee3,1116 Walnut St,...,538,"[1116 Walnut St (btwn 11th & 12th St), Philade...","[{'label': 'display', 'lat': 39.94878527635662...",39.948785,-75.159855,19107,PA,Moriarty's Restaurant & Irish Pub,v-1581876982,NaN
1,"[{'id': '4bf58dd8d48988d147941735', 'name': 'D...",365418,/delivery_provider_grubhub_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",grubhub,https://www.grubhub.com/restaurant/continental...,False,4a620924f964a520dfc21fe3,28 S 18th St,...,580,"[28 S 18th St, Philadelphia, PA 19103, United ...","[{'label': 'display', 'lat': 39.95225354521868...",39.952254,-75.170296,19103,PA,Midtown III Restaurant,v-1581876982,NaN
2,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",559902,/delivery_provider_grubhub_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",grubhub,https://www.grubhub.com/restaurant/e-mei-resta...,False,4d85387c61676dcb586b85e4,915 Arch St,...,697,"[915 Arch St, Philadelphia, PA 19107, United S...","[{'label': 'display', 'lat': 39.95360784404011...",39.953608,-75.155434,19107,PA,E Mei Restaurant,v-1581876982,418158388
3,"[{'id': '4bf58dd8d48988d11d941735', 'name': 'S...",NaN,NaN,NaN,NaN,NaN,NaN,False,49f124b5f964a52090691fe3,39-41 S 19th St,...,707,[39-41 S 19th St (btwn Chestnut St & Market St...,"[{'label': 'display', 'lat': 39.95223521835672...",39.952235,-75.171792,19103,PA,Smiths Restaurant & Bar,v-1581876982,82045860
4,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",NaN,NaN,NaN,NaN,NaN,NaN,False,4a776a0df964a52068e41fe3,210 W Rittenhouse Sq,...,852,"[210 W Rittenhouse Sq, Philadelphia, PA 19103,...","[{'label': 'display', 'lat': 39.94968254511642...",39.949683,-75.172693,19103,PA,Lacroix Restaurant at The Rittenhouse,v-1581876982,80759178


In [21]:
# keep only columns that include venue name, and anything that is associated with location
Restaurant_clean_columns = ['name', 'categories'] + [col for col in Restaurant_dataframe.columns if col.startswith('location.')]+ ['id']
clean_Restaurant_dataframe = Restaurant_dataframe.loc[:,Restaurant_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list3 = row['categories']
    except:
        categories_list3 = row['venue.categories']
        
    if len(categories_list3) == 0:
        return None
    else:
        return categories_list3[0]['name']

# filter the category for each row
clean_Restaurant_dataframe['categories'] = clean_Restaurant_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_Restaurant_dataframe.columns = [column.split('.')[-1] for column in clean_Restaurant_dataframe.columns]

clean_Restaurant_dataframe.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Moriarty's Restaurant & Irish Pub,Pub,1116 Walnut St,US,Philadelphia,United States,btwn 11th & 12th St,538,"[1116 Walnut St (btwn 11th & 12th St), Philade...","[{'label': 'display', 'lat': 39.94878527635662...",39.948785,-75.159855,19107,PA,40b28c80f964a52033f91ee3
1,Midtown III Restaurant,Diner,28 S 18th St,US,Philadelphia,United States,NaN,580,"[28 S 18th St, Philadelphia, PA 19103, United ...","[{'label': 'display', 'lat': 39.95225354521868...",39.952254,-75.170296,19103,PA,4a620924f964a520dfc21fe3
2,E Mei Restaurant,Chinese Restaurant,915 Arch St,US,Philadelphia,United States,NaN,697,"[915 Arch St, Philadelphia, PA 19107, United S...","[{'label': 'display', 'lat': 39.95360784404011...",39.953608,-75.155434,19107,PA,4d85387c61676dcb586b85e4
3,Smiths Restaurant & Bar,Sports Bar,39-41 S 19th St,US,Philadelphia,United States,btwn Chestnut St & Market St,707,[39-41 S 19th St (btwn Chestnut St & Market St...,"[{'label': 'display', 'lat': 39.95223521835672...",39.952235,-75.171792,19103,PA,49f124b5f964a52090691fe3
4,Lacroix Restaurant at The Rittenhouse,Restaurant,210 W Rittenhouse Sq,US,Philadelphia,United States,NaN,852,"[210 W Rittenhouse Sq, Philadelphia, PA 19103,...","[{'label': 'display', 'lat': 39.94968254511642...",39.949683,-75.172693,19103,PA,4a776a0df964a52068e41fe3


In [22]:
# delete unnecessary columns
clean_Restaurant_dataframe2= clean_Restaurant_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'id'], axis=1)
clean_Restaurant_dataframe2

,name,categories,address,lat,lng,postalCode,state
0,Moriarty's Restaurant & Irish Pub,Pub,1116 Walnut St,39.948785,-75.159855,19107,PA
1,Midtown III Restaurant,Diner,28 S 18th St,39.952254,-75.170296,19103,PA
2,E Mei Restaurant,Chinese Restaurant,915 Arch St,39.953608,-75.155434,19107,PA
3,Smiths Restaurant & Bar,Sports Bar,39-41 S 19th St,39.952235,-75.171792,19103,PA
4,Lacroix Restaurant at The Rittenhouse,Restaurant,210 W Rittenhouse Sq,39.949683,-75.172693,19103,PA
5,Vietnam Restaurant,Vietnamese Restaurant,221 N 11th St,39.956300,-75.157318,19107,PA
6,Ocean City Seafood Restaurant,Dim Sum Restaurant,234 N 9th St,39.956269,-75.154141,19107,PA
7,North Third Restaurant,American Restaurant,801 N 3rd St,39.963445,-75.142862,19123,PA
8,Iron Hill Brewery & Restaurant,Brewery,1150 Market St,39.951840,-75.159455,19107,PA
9,M Restaurant,American Restaurant,231 S 8th St,39.947301,-75.154198,19106,PA


In [23]:
# delete rows with none values
df_Restaurant = clean_Restaurant_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
df_Restaurant

,name,categories,address,lat,lng,postalCode,state
0,Moriarty's Restaurant & Irish Pub,Pub,1116 Walnut St,39.948785,-75.159855,19107,PA
1,Midtown III Restaurant,Diner,28 S 18th St,39.952254,-75.170296,19103,PA
2,E Mei Restaurant,Chinese Restaurant,915 Arch St,39.953608,-75.155434,19107,PA
3,Smiths Restaurant & Bar,Sports Bar,39-41 S 19th St,39.952235,-75.171792,19103,PA
4,Lacroix Restaurant at The Rittenhouse,Restaurant,210 W Rittenhouse Sq,39.949683,-75.172693,19103,PA
5,Vietnam Restaurant,Vietnamese Restaurant,221 N 11th St,39.956300,-75.157318,19107,PA
6,Ocean City Seafood Restaurant,Dim Sum Restaurant,234 N 9th St,39.956269,-75.154141,19107,PA
7,North Third Restaurant,American Restaurant,801 N 3rd St,39.963445,-75.142862,19123,PA
8,Iron Hill Brewery & Restaurant,Brewery,1150 Market St,39.951840,-75.159455,19107,PA
9,M Restaurant,American Restaurant,231 S 8th St,39.947301,-75.154198,19106,PA


# Search for Cafeteria

In [24]:
# search for Cafeteria
search_query = 'Cafeteria'
radius = 10000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius)
url

'https://api.foursquare.com/v2/venues/search?client_id=GEOB3Q4RTF3RYS1WW0YUO1K1TH2S3JACF40KFZGGILESXARV&client_secret=DZT1HG4VZQFFQVQRZLIVQ25HFWJAEAWIEKKO2AYGE1HHOB3J&ll=39.9527237,-75.1635262&v=20200212&query=Cafeteria&radius=10000'

In [25]:
# Send the GET Request and examine the results
cresults = requests.get(url).json()
#cresults

In [26]:

# assign relevant part of JSON to venues
venues = cresults['response']['venues']

# tranform venues into a dataframe
Cafeteria_dataframe = json_normalize(venues)
Cafeteria_dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1a1941735', 'name': 'C...",False,4c98ca9b4804a143d584ee0e,Bonnell Building,US,Philadelphia,United States,Community College of Philadelphia,1048,[Bonnell Building (Community College of Philad...,"[{'label': 'display', 'lat': 39.96197657816427...",39.961977,-75.165810,NaN,19130,PA,Cafeteria,v-1581876982,NaN
1,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",False,50fc34d8e4b0937c7ee94987,Philadelphia Museum of Art,US,Philadelphia,United States,1st Floor,2062,"[Philadelphia Museum of Art (1st Floor), Phila...","[{'label': 'display', 'lat': 39.96509849221878...",39.965098,-75.181517,NaN,19130,PA,Art Museum Cafeteria,v-1581876982,NaN
2,"[{'id': '54f4ba06498e2cf5561da814', 'name': 'C...",False,5bd1ef8c46e1b6002cc08649,1800 Arch St,US,Philadelphia,United States,NaN,610,"[1800 Arch St, Philadelphia, PA 19103, United ...","[{'label': 'display', 'lat': 39.95494946560051...",39.954949,-75.170067,Logan Square,19103,PA,CTC Cafeteria,v-1581876982,NaN
3,"[{'id': '4bf58dd8d48988d1a1941735', 'name': 'C...",False,512cfb27e4b0c7118ac65b0a,1025 Walnut St,US,Philadelphia,United States,NaN,610,"[1025 Walnut St, Philadelphia, PA 19107, Unite...","[{'label': 'display', 'lat': 39.949461, 'lng':...",39.949461,-75.157773,NaN,19107,PA,Jefferson Alumni Hall Cafeteria,v-1581876982,NaN
4,"[{'id': '4bf58dd8d48988d1a1941735', 'name': 'C...",False,522df244498edbef4f579cef,NaN,US,Philadelphia,United States,NaN,250,"[Philadelphia, PA, United States]","[{'label': 'display', 'lat': 39.952316, 'lng':...",39.952316,-75.160635,NaN,NaN,PA,Claims College Cafeteria,v-1581876982,NaN


In [27]:

# keep only columns that include venue name, and anything that is associated with location
Cafeteria_clean_columns = ['name', 'categories'] + [col for col in Cafeteria_dataframe.columns if col.startswith('location.')]+ ['id']
clean_Cafeteria_dataframe = Cafeteria_dataframe.loc[:,Cafeteria_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list4 = row['categories']
    except:
        categories_list4 = row['venue.categories']
        
    if len(categories_list4) == 0:
        return None
    else:
        return categories_list4[0]['name']

# filter the category for each row
clean_Cafeteria_dataframe['categories'] = clean_Cafeteria_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_Cafeteria_dataframe.columns = [column.split('.')[-1] for column in clean_Cafeteria_dataframe.columns]

clean_Cafeteria_dataframe.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Cafeteria,College Cafeteria,Bonnell Building,US,Philadelphia,United States,Community College of Philadelphia,1048,[Bonnell Building (Community College of Philad...,"[{'label': 'display', 'lat': 39.96197657816427...",39.961977,-75.165810,NaN,19130,PA,4c98ca9b4804a143d584ee0e
1,Art Museum Cafeteria,Café,Philadelphia Museum of Art,US,Philadelphia,United States,1st Floor,2062,"[Philadelphia Museum of Art (1st Floor), Phila...","[{'label': 'display', 'lat': 39.96509849221878...",39.965098,-75.181517,NaN,19130,PA,50fc34d8e4b0937c7ee94987
2,CTC Cafeteria,Corporate Cafeteria,1800 Arch St,US,Philadelphia,United States,NaN,610,"[1800 Arch St, Philadelphia, PA 19103, United ...","[{'label': 'display', 'lat': 39.95494946560051...",39.954949,-75.170067,Logan Square,19103,PA,5bd1ef8c46e1b6002cc08649
3,Jefferson Alumni Hall Cafeteria,College Cafeteria,1025 Walnut St,US,Philadelphia,United States,NaN,610,"[1025 Walnut St, Philadelphia, PA 19107, Unite...","[{'label': 'display', 'lat': 39.949461, 'lng':...",39.949461,-75.157773,NaN,19107,PA,512cfb27e4b0c7118ac65b0a
4,Claims College Cafeteria,College Cafeteria,NaN,US,Philadelphia,United States,NaN,250,"[Philadelphia, PA, United States]","[{'label': 'display', 'lat': 39.952316, 'lng':...",39.952316,-75.160635,NaN,NaN,PA,522df244498edbef4f579cef


In [28]:
# delete unnecessary columns
clean_Cafeteria_dataframe2= clean_Cafeteria_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'id'], axis=1)
clean_Cafeteria_dataframe2

,name,categories,address,lat,lng,neighborhood,postalCode,state
0,Cafeteria,College Cafeteria,Bonnell Building,39.961977,-75.165810,NaN,19130,PA
1,Art Museum Cafeteria,Café,Philadelphia Museum of Art,39.965098,-75.181517,NaN,19130,PA
2,CTC Cafeteria,Corporate Cafeteria,1800 Arch St,39.954949,-75.170067,Logan Square,19103,PA
3,Jefferson Alumni Hall Cafeteria,College Cafeteria,1025 Walnut St,39.949461,-75.157773,NaN,19107,PA
4,Claims College Cafeteria,College Cafeteria,NaN,39.952316,-75.160635,NaN,NaN,PA
5,Basement H&M Employee Cafeteria,Clothing Store,NaN,39.951385,-75.167020,NaN,19102,PA
6,Cafeteria,College Cafeteria,Cooper University Hospital,39.941613,-75.116173,NaN,08103,NJ
7,Hahnemann University Hospital Cafeteria,Cafeteria,NaN,39.957368,-75.162932,NaN,NaN,PA
8,Jefferson Hospital Atrium Cafeteria,Cafeteria,110 S. 10th St.,39.949971,-75.158166,NaN,19107,PA
9,Sodexo Cafeteria @ GSK,American Restaurant,200 N 16th St,39.957088,-75.166235,NaN,19102,PA


In [29]:
# delete rows with none values
df_Cafeteria = clean_Cafeteria_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
df_Cafeteria

,name,categories,address,lat,lng,neighborhood,postalCode,state
2,CTC Cafeteria,Corporate Cafeteria,1800 Arch St,39.954949,-75.170067,Logan Square,19103,PA


# Search for Shopping stories

In [30]:
# search for Shopping
search_query = 'Shopping'
radius = 1000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius)
url

'https://api.foursquare.com/v2/venues/search?client_id=GEOB3Q4RTF3RYS1WW0YUO1K1TH2S3JACF40KFZGGILESXARV&client_secret=DZT1HG4VZQFFQVQRZLIVQ25HFWJAEAWIEKKO2AYGE1HHOB3J&ll=39.9527237,-75.1635262&v=20200212&query=Shopping&radius=1000'

In [31]:
# Send the GET Request and examine the results
sresults = requests.get(url).json()
#sresults

In [32]:
# assign relevant part of JSON to venues
venues = sresults['response']['venues']

# tranform venues into a dataframe
Shopping_dataframe = json_normalize(venues)
Shopping_dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",False,4fc8fa70e4b0ccf717c72c64,NaN,US,Philadelphia,United States,447,"[Philadelphia, PA 19103, United States]","[{'label': 'display', 'lat': 39.94973955025363...",39.949740,-75.167047,19103,PA,Walnut Street Shopping,v-1581876982
1,"[{'id': '4bf58dd8d48988d107951735', 'name': 'S...",False,55bef1b8498ec078805c74b1,1509 Walnut St,US,Philadelphia,United States,401,"[1509 Walnut St, Philadelphia, PA 19102, Unite...","[{'label': 'display', 'lat': 39.949843, 'lng':...",39.949843,-75.166356,19102,PA,On-Line Shopping Store,v-1581876982
2,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O...",False,55af9310498e5120ee5f91cf,1217 Sansom St Fl 8,US,Philadelphia,United States,377,"[1217 Sansom St Fl 8, Philadelphia, PA 19107, ...","[{'label': 'display', 'lat': 39.94984799293115...",39.949848,-75.161183,19107,PA,PayrollShopping.com,v-1581876982


In [33]:
# keep only columns that include venue name, and anything that is associated with location
Shopping_clean_columns = ['name', 'categories'] + [col for col in Shopping_dataframe.columns if col.startswith('location.')]+ ['id']
clean_Shopping_dataframe = Shopping_dataframe.loc[:,Shopping_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list5 = row['categories']
    except:
        categories_list5 = row['venue.categories']
        
    if len(categories_list5) == 0:
        return None
    else:
        return categories_list5[0]['name']

# filter the category for each row
clean_Shopping_dataframe['categories'] = clean_Shopping_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_Shopping_dataframe.columns = [column.split('.')[-1] for column in clean_Shopping_dataframe.columns]

clean_Shopping_dataframe.head()

,name,categories,address,cc,city,country,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Walnut Street Shopping,Shopping Mall,NaN,US,Philadelphia,United States,447,"[Philadelphia, PA 19103, United States]","[{'label': 'display', 'lat': 39.94973955025363...",39.949740,-75.167047,19103,PA,4fc8fa70e4b0ccf717c72c64
1,On-Line Shopping Store,Shoe Store,1509 Walnut St,US,Philadelphia,United States,401,"[1509 Walnut St, Philadelphia, PA 19102, Unite...","[{'label': 'display', 'lat': 39.949843, 'lng':...",39.949843,-75.166356,19102,PA,55bef1b8498ec078805c74b1
2,PayrollShopping.com,Office,1217 Sansom St Fl 8,US,Philadelphia,United States,377,"[1217 Sansom St Fl 8, Philadelphia, PA 19107, ...","[{'label': 'display', 'lat': 39.94984799293115...",39.949848,-75.161183,19107,PA,55af9310498e5120ee5f91cf


In [34]:
# delete unnecessary columns
clean_Shopping_dataframe2= clean_Shopping_dataframe.drop(['cc', 'city', 'country', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'id'], axis=1)
clean_Shopping_dataframe2

,name,categories,address,lat,lng,postalCode,state
0,Walnut Street Shopping,Shopping Mall,NaN,39.949740,-75.167047,19103,PA
1,On-Line Shopping Store,Shoe Store,1509 Walnut St,39.949843,-75.166356,19102,PA
2,PayrollShopping.com,Office,1217 Sansom St Fl 8,39.949848,-75.161183,19107,PA


In [35]:

# delete rows which its category is not Shopping Mall
df_Shopping = clean_Shopping_dataframe2[clean_Shopping_dataframe2.categories == 'Shopping Mall']
df_Shopping

,name,categories,address,lat,lng,postalCode,state
0,Walnut Street Shopping,Shopping Mall,NaN,39.94974,-75.167047,19103,PA


# Search for Parks

In [36]:
# search for Parks
search_query = 'Park'
radius = 10000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}'\
.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius)
url

'https://api.foursquare.com/v2/venues/search?client_id=GEOB3Q4RTF3RYS1WW0YUO1K1TH2S3JACF40KFZGGILESXARV&client_secret=DZT1HG4VZQFFQVQRZLIVQ25HFWJAEAWIEKKO2AYGE1HHOB3J&ll=39.9527237,-75.1635262&v=20200212&query=Park&radius=10000'

In [37]:

# Send the GET Request and examine the results
presults = requests.get(url).json()
#presults

In [38]:
# assign relevant part of JSON to venues
venues = presults['response']['venues']

# tranform venues into a dataframe
park_dataframe = json_normalize(venues)
park_dataframe.head()

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,...,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d164941735', 'name': 'P...",NaN,NaN,NaN,NaN,NaN,NaN,False,4aa14edef964a5206d4020e3,1501 John F Kennedy Blvd,...,217,"[1501 John F Kennedy Blvd (at N 15th St), Phil...",NaN,39.954123,-75.165303,NaN,19102,PA,JFK Plaza / Love Park,v-1581876983
1,"[{'id': '52e81612bcbc57f1066b7a21', 'name': 'N...",NaN,NaN,NaN,NaN,NaN,NaN,False,4b6c740af964a520683b2ce3,143 S 3rd St,...,1194,"[143 S 3rd St (btwn Race & Walnut St), Philade...","[{'label': 'display', 'lat': 39.95066565989744...",39.950666,-75.149787,NaN,19106,PA,Independence National Historical Park,v-1581876983
2,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",NaN,NaN,NaN,NaN,NaN,NaN,False,4bde0d566198c9b6c5cc12ff,1 Penn Sq,...,102,"[1 Penn Sq (West Side of City Hall), Philadelp...","[{'label': 'display', 'lat': 39.95277184634310...",39.952772,-75.164723,Avenue of the Arts South,19102,PA,Dilworth Park,v-1581876983
3,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",NaN,NaN,NaN,NaN,NaN,NaN,False,4c12402077cea5936aa7cc60,Logan Circle,...,757,"[Logan Circle (at Ben Franklin Pkwy), Philadel...","[{'label': 'display', 'lat': 39.95785953028045...",39.957860,-75.169344,NaN,19103,PA,Sister Cities Park,v-1581876983
4,"[{'id': '4c38df4de52ce0d596b336e1', 'name': 'P...",NaN,NaN,NaN,NaN,NaN,NaN,False,520d435011d2d981c69d2125,123 S 12th St,...,326,"[123 S 12th St (at Sansom St), Philadelphia, P...","[{'label': 'display', 'lat': 39.95043650206751...",39.950437,-75.161136,Center City East,19107,PA,"Park America, Inc.",v-1581876983


In [39]:
# keep only columns that include venue name, and anything that is associated with location
park_clean_columns = ['name', 'categories'] + [col for col in park_dataframe.columns if col.startswith('location.')]+ ['id']
clean_park_dataframe = park_dataframe.loc[:,park_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list1 = row['categories']
    except:
        categories_list1 = row['venue.categories']
        
    if len(categories_list1) == 0:
        return None
    else:
        return categories_list1[0]['name']

# filter the category for each row
clean_park_dataframe['categories'] = clean_park_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_park_dataframe.columns = [column.split('.')[-1] for column in clean_park_dataframe.columns]

clean_park_dataframe.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,JFK Plaza / Love Park,Plaza,1501 John F Kennedy Blvd,US,Philadelphia,United States,at N 15th St,217,"[1501 John F Kennedy Blvd (at N 15th St), Phil...",NaN,39.954123,-75.165303,NaN,19102,PA,4aa14edef964a5206d4020e3
1,Independence National Historical Park,National Park,143 S 3rd St,US,Philadelphia,United States,btwn Race & Walnut St,1194,"[143 S 3rd St (btwn Race & Walnut St), Philade...","[{'label': 'display', 'lat': 39.95066565989744...",39.950666,-75.149787,NaN,19106,PA,4b6c740af964a520683b2ce3
2,Dilworth Park,Park,1 Penn Sq,US,Philadelphia,United States,West Side of City Hall,102,"[1 Penn Sq (West Side of City Hall), Philadelp...","[{'label': 'display', 'lat': 39.95277184634310...",39.952772,-75.164723,Avenue of the Arts South,19102,PA,4bde0d566198c9b6c5cc12ff
3,Sister Cities Park,Park,Logan Circle,US,Philadelphia,United States,at Ben Franklin Pkwy,757,"[Logan Circle (at Ben Franklin Pkwy), Philadel...","[{'label': 'display', 'lat': 39.95785953028045...",39.957860,-75.169344,NaN,19103,PA,4c12402077cea5936aa7cc60
4,"Park America, Inc.",Parking,123 S 12th St,US,Philadelphia,United States,at Sansom St,326,"[123 S 12th St (at Sansom St), Philadelphia, P...","[{'label': 'display', 'lat': 39.95043650206751...",39.950437,-75.161136,Center City East,19107,PA,520d435011d2d981c69d2125


In [40]:

# delete unnecessary columns
clean_park_dataframe2= clean_park_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'id'], axis=1)
clean_park_dataframe2

,name,categories,address,lat,lng,neighborhood,postalCode,state
0,JFK Plaza / Love Park,Plaza,1501 John F Kennedy Blvd,39.954123,-75.165303,NaN,19102,PA
1,Independence National Historical Park,National Park,143 S 3rd St,39.950666,-75.149787,NaN,19106,PA
2,Dilworth Park,Park,1 Penn Sq,39.952772,-75.164723,Avenue of the Arts South,19102,PA
3,Sister Cities Park,Park,Logan Circle,39.957860,-75.169344,NaN,19103,PA
4,"Park America, Inc.",Parking,123 S 12th St,39.950437,-75.161136,Center City East,19107,PA
5,Ned Smyth World Park,Park,N 12 St,39.952924,-75.160157,Center City East,19107,PA
6,Louis I. Kahn Memorial Park,Park,11th St. & Pine St.,39.945092,-75.159960,NaN,19107,PA
7,Fairmount Park,Park,N Concourse St,39.982903,-75.211187,NaN,19131,PA
8,Towne Park Valet,Parking,1200 Market St,39.951166,-75.159994,NaN,19107,PA
9,Matthias Baldwin Park,Park,Franklin Town Blvd & Callowhill St,39.961497,-75.169026,NaN,19130,PA


In [41]:
# delete rows with none values
clean_park_dataframe3 = clean_park_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
clean_park_dataframe3

,name,categories,address,lat,lng,neighborhood,postalCode,state
2,Dilworth Park,Park,1 Penn Sq,39.952772,-75.164723,Avenue of the Arts South,19102,PA
4,"Park America, Inc.",Parking,123 S 12th St,39.950437,-75.161136,Center City East,19107,PA
5,Ned Smyth World Park,Park,N 12 St,39.952924,-75.160157,Center City East,19107,PA
14,Rothman Institute Ice Rink at Dilworth Park,Skating Rink,1 Penn Sq,39.952770,-75.164561,Avenue of the Arts South,19102,PA
17,Liberty Lands Park,Park,N 3rd St,39.965609,-75.142045,Northern Liberties,19123,PA


In [42]:
# delete rows which its category is not Park
df_park = clean_park_dataframe3[clean_park_dataframe3.categories == 'Park']
df_park

,name,categories,address,lat,lng,neighborhood,postalCode,state
2,Dilworth Park,Park,1 Penn Sq,39.952772,-75.164723,Avenue of the Arts South,19102,PA
5,Ned Smyth World Park,Park,N 12 St,39.952924,-75.160157,Center City East,19107,PA
17,Liberty Lands Park,Park,N 3rd St,39.965609,-75.142045,Northern Liberties,19123,PA


# Generate map to visualize hotels, shopping stores and Cafeteria and how they cluster together

In [43]:

# create dataframe of hotels, shopping stores and Cafeteria
hotel_neighbourhood_df = pd.concat([clean_grocerystore_dataframe3, df_hotels, df_Cafeteria, df_Shopping], ignore_index=True)
hotel_neighbourhood_df

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  app.launch_new_instance()


,address,categories,lat,lng,name,neighborhood,postalCode,state
0,1227 S 18th St,Grocery Store,39.935849,-75.173754,Fernandez Grocery Store,NaN,19146,PA
1,2200 Benjamin Franklin Pkwy,Grocery Store,39.958787,-75.175407,Grocery Store,NaN,19130,PA
2,1500 S Broad St,Food & Drink Shop,39.931673,-75.168482,88 Grocery Store,NaN,19146,PA
3,7th Street,Grocery Store,39.922412,-75.158218,Eak Meng: Cambodian Grocery Store,NaN,19148,PA
4,115 W Girard Ave,Middle Eastern Restaurant,39.969112,-75.136764,Jerusalem Grocery Store,NaN,19123,PA
5,South Camac St.,Grocery Store,39.916994,-75.168181,Venuti's Grocery Store,NaN,19148,PA
6,1000 Snyder Ave,Grocery Store,39.923217,-75.162336,10th & Snyder Grocery Store,NaN,19148,PA
7,17th Street,Mexican Restaurant,39.964811,-75.165907,Grocery Stores,NaN,19130,PA
8,2200 S 20th St,Deli / Bodega,39.924285,-75.179552,Anna Grocery Store,NaN,19145,PA
9,400 McKean Street,Deli / Bodega,39.923715,-75.153132,Andrew Grocery Store,NaN,19146,PA


In [44]:

# Generate map to visualize hotel neighbourhood including shopping stores and Cafeteria 
hotel_map = folium.Map(location=[latitude, longitude], zoom_start=14)

for lat, lng, name, categories, address in zip(hotel_neighbourhood_df['lat'], hotel_neighbourhood_df['lng'], 
                                           hotel_neighbourhood_df['name'], hotel_neighbourhood_df['categories'],\
                                               hotel_neighbourhood_df['address']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(hotel_map)  
    
hotel_map

# Generate map to visualize Park, Restaurant and Cafeteria and how they cluster together

In [45]:
# create dataframe of Park, Restaurant and Cafeteria
park_neighbourhood_df = pd.concat([df_park, df_Restaurant, df_Cafeteria,], ignore_index=True)
park_neighbourhood_df

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  from ipykernel import kernelapp as app


,address,categories,lat,lng,name,neighborhood,postalCode,state
0,1 Penn Sq,Park,39.952772,-75.164723,Dilworth Park,Avenue of the Arts South,19102,PA
1,N 12 St,Park,39.952924,-75.160157,Ned Smyth World Park,Center City East,19107,PA
2,N 3rd St,Park,39.965609,-75.142045,Liberty Lands Park,Northern Liberties,19123,PA
3,1116 Walnut St,Pub,39.948785,-75.159855,Moriarty's Restaurant & Irish Pub,NaN,19107,PA
4,28 S 18th St,Diner,39.952254,-75.170296,Midtown III Restaurant,NaN,19103,PA
5,915 Arch St,Chinese Restaurant,39.953608,-75.155434,E Mei Restaurant,NaN,19107,PA
6,39-41 S 19th St,Sports Bar,39.952235,-75.171792,Smiths Restaurant & Bar,NaN,19103,PA
7,210 W Rittenhouse Sq,Restaurant,39.949683,-75.172693,Lacroix Restaurant at The Rittenhouse,NaN,19103,PA
8,221 N 11th St,Vietnamese Restaurant,39.956300,-75.157318,Vietnam Restaurant,NaN,19107,PA
9,234 N 9th St,Dim Sum Restaurant,39.956269,-75.154141,Ocean City Seafood Restaurant,NaN,19107,PA


In [46]:
# Generate map to visualize park neighbourhood including Restaurant and Cafeteria 
park_map = folium.Map(location=[latitude, longitude], zoom_start=14)

for lat, lng, name, categories, address in zip(park_neighbourhood_df['lat'], park_neighbourhood_df['lng'], 
                                           park_neighbourhood_df['name'], park_neighbourhood_df['categories'],\
                                               park_neighbourhood_df['address']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(park_map)  
    
park_map